<a href="https://colab.research.google.com/github/Dstratigou/Askhsh/blob/main/%CE%91%CE%BD%CF%84%CE%AF%CE%B3%CF%81%CE%B1%CF%86%CE%BF_%CE%91%CE%BD%CF%84%CE%AF%CE%B3%CF%81%CE%B1%CF%86%CE%BF_%CE%91%CE%BD%CF%84%CE%AF%CE%B3%CF%81%CE%B1%CF%86%CE%BF_stratigou_praktikh_efarmogh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#εισαγωγή βιβλιοθηκών - πακέτων
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTENC
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.compose import ColumnTransformer
import requests
from io import BytesIO
import numpy as np

In [ ]:
#κατέβασμα  του συνόλου δεδομένων
url = "https://github.com/DSstratigou/DSstratigou/blob/main/DarkNet.csv"
response = requests.get(url, stream=True)


In [ ]:
#φόρτωση του συνόλου δεδομένων
columns = [ "Src_IP","Src_Port","Dst_IP","Dst_Port","Protocol","Flow_Duration","Total_Fwd_Packet","Total_Bwd_packets","Total_Length_of_Fwd_Packet",
"Total_Length_of_Bwd_Packet","Fwd_Packet_Length_Max","Fwd_Packet_Length_Min","Fwd_Packet_Length_Mean","Fwd_Packet_Length_Std",
"Bwd_Packet_Length_Max","Bwd_Packet_Length_Min","Bwd_Packet_Length_Mean",
"Bwd_Packet_Length_Std","Flow_Bytes/s","Flow_Packets/s","Flow_IAT_Mean","Flow_IAT_Std","Flow_IAT_Max","Flow_IAT_Min","Fwd_IAT_Total","Fwd_IAT_Mean",
"Fwd_IAT_Std","Fwd_IAT_Max","Fwd_IAT_Min","Bwd_IAT_Total","Bwd_IAT_Mean","Bwd_IAT_Std","Bwd_IAT_Max","Bwd_IAT_Min","Fwd_PSH_Flags","Bwd_PSH_Flags",
"Fwd_URG_Flags","Bwd_URG_Flags","Fwd_Header_Length","Bwd_Header_Length","Fwd_Packets/s","Bwd_Packets/s","Packet_Length_Min","Packet_Length_Max",
"Packet_Length_Mean","Packet_Length_Std","Packet_Length_Variance","FIN_Flag_Count","SYN_Flag_Count","RST_Flag_Count","PSH_Flag_Count","ACK_Flag_Count",
"URG_Flag_Count","CWE_Flag_Count","ECE_Flag_Count","Down/Up_Ratio","Average_Packet_Size","Fwd_Segment_Size_Avg","Bwd_Segment_Size_Avg",
"Fwd_Bytes/Bulk_Avg","Fwd_Packet/Bulk_Avg","Fwd_Bulk_Rate_Avg","Bwd_Bytes/Bulk_Avg","Bwd_Packet/Bulk_Avg","Bwd_Bulk_Rate_Avg","Subflow_Fwd_Packets",
"Subflow_Fwd_Bytes","Subflow_Bwd_Packets","Subflow_Bwd_Bytes","FWD_Init_Win_Bytes","Bwd_Init_Win_Bytes","Fwd_Act_Data_Pkts","Fwd_Seg_Size_Min",
"Active_Mean","Active_Std","Active_Max","Active_Min","Idle_Mean","Idle_Std","Idle_Max","Idle_Min","Label-1","Label-2"]

df = pd.read_csv(response, header=None, names=columns)


ValueError: Invalid file path or buffer object type: <class 'requests.models.Response'>

In [ ]:
#δημιουργία του συνόλου σε δύο κλάσεις Non-Tor και AUDIO-STREAMING
X = df.drop(["Label-1","Label-2"], axis=1)
y = df["Label-1","Label-2"].apply(lambda x: 0 if x == "normal." else 1)

NameError: name 'df' is not defined

In [ ]:
#έλεγχος κλάσεων
print("Κατανομή κλάσεων y:")
print(y.value_counts())

Κατανομή κλάσεων y:


NameError: name 'y' is not defined

In [ ]:
#δημιουργία διοχέτευσης (αγωγού) προεπεξεργασίας με κωδικοποίηση μίας δέσμης (one-hot encoding) για κατηγορικές μεταβλητές
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),

    ])

NameError: name 'numeric_features' is not defined

In [ ]:
#τμηματοποίηση δεδομένων σε σύνολα εκαπίδευσης και δοκιμών
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

NameError: name 'X' is not defined

In [ ]:
#δημιουργία διοχέτευσης (αγωγού) SMOTE για τις αριθμητικές μεταβλητές μόνο
pipeline = ImbPipeline([
    ('preprocessor', preprocessor),

    ('classifier', RandomForestClassifier(random_state=42))
])

NameError: name 'preprocessor' is not defined

In [ ]:
#καθορισμός κατωφλίου (threshold) για την διακοπή διακτυακής κίνησης
blocking_threshold = 0.9

In [ ]:
#δημιουργία βρόχου συνεχούς - αυξητικής μάθησης
batch_size = 10000
for epoch in range(1, 3):  #δυνατότητα αλλαγής των εποχών
    for i in range(0, len(X_train), batch_size):
        X_batch = X_train.iloc[i:i + batch_size]
        y_batch = y_train.iloc[i:i + batch_size]

        #σταδιακή ενημέρωση του μοντέλου με κάθε ροή (batch) δεδομένων
        pipeline.fit(X_batch, y_batch)

        #περιοδική ενημέρωση του μοντέλου στο σύνολο δοκιμών
        if i % batch_size == 0 and i > 0:
            y_pred_proba = pipeline.predict_proba(X_test)[:, 1]

            #αποκλεισμός δικτυακής κυκλοφορίας εάν η προβλεπόμενη πιθανότητα υπερβαίνει το καθορισμένο όριο
            blocked_indices = np.where(y_pred_proba > blocking_threshold)[0]
            if len(blocked_indices) > 0:
                print(f"Blocking {len(blocked_indices)} malicious traffic instances.")

            accuracy = accuracy_score(y_test, y_pred_proba > blocking_threshold)
            print(f"Epoch {epoch}, Iteration {i}, Test Accuracy: {accuracy}")

NameError: name 'X_train' is not defined

In [ ]:
#τελική αξιολόγηση του μοντέλου στο σύνολο δοκιμών
y_pred_proba = pipeline.predict_proba(X_test)[:, 1]
blocked_indices = np.where(y_pred_proba > blocking_threshold)[0]
if len(blocked_indices) > 0:
    print(f"Blocking {len(blocked_indices)} malicious traffic instances.")

accuracy = accuracy_score(y_test, y_pred_proba > blocking_threshold)
classification_rep = classification_report(y_test, y_pred_proba > blocking_threshold)

#εκτύπωση τελικών αποτελεσμάτων
print(f"Final Test Accuracy: {accuracy}")
print("Classification Report:")
print(classification_rep)

NameError: name 'pipeline' is not defined